Орехова Дарья, 15 ФПЛ

Задача №16: 1,9x^3 – 2,9x^3 – x^3

In [256]:
import numpy as np

In [257]:
class Operation():  
    
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes
        self.output_nodes = []        
        for node in input_nodes:
            node.output_nodes.append(self)        
        _default_graph.operations.append(self)
  
    def compute(self):                
        pass

In [258]:
class add(Operation): 
    
    def __init__(self, x, y):         
        super().__init__([x, y])

    def compute(self, x_var, y_var):         
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [259]:
class multiply(Operation):
    
    def __init__(self, a, b):        
        super().__init__([a, b])
        
    def compute(self, a_var, b_var):         
        self.inputs = [a_var, b_var]
        return a_var * b_var

In [260]:
class Placeholder():
        
    def __init__(self):        
        self.output_nodes = []        
        _default_graph.placeholders.append(self)

In [261]:
class Variable():
        
    def __init__(self, initial_value = None):        
        self.value = initial_value
        self.output_nodes = []        
        _default_graph.variables.append(self)

In [262]:
class Graph():
       
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [263]:
g = Graph()
g.set_as_default()
a = Variable(1.9)
b = Variable(-2.9)
c = Variable(-1)

x = Placeholder()

d = multiply(x, x) #x^2
e = multiply(x, d) #x^3
f = multiply(a, e) #1,9x^3
g = multiply(b, e) #-2,9x^3
h = add(f, g) #1,9x^3-2,9x^3
i = multiply(c, e) #-1x^3
j = add(h, i) #1,9x^3-2,9x^3-1x^3

In [264]:
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [265]:
class Session:
    
    def run(self, operation, feed_dict = {}):        
        nodes_postorder = traverse_postorder(operation)        
        for node in nodes_postorder:
            if type(node) == Placeholder:                
                node.output = feed_dict[node]                
            elif type(node) == Variable:                
                node.output = node.value                
            else:                     
                node.inputs = [input_node.output for input_node in node.input_nodes]                 
                node.output = node.compute(*node.inputs)                
            if type(node.output) == list:
                node.output = np.array(node.output)        
        return operation.output

In [266]:
sess = Session()    
result = sess.run(operation=j,feed_dict={x: [3, 4, 5]})    

In [267]:
result

array([ -54., -128., -250.])